In [10]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery

In [9]:

query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


In [11]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9989.000000,10000.000000,10000.000000,9890.000000
mean,7.297602,27.298900,1.034400,38.699798
std,1.291685,6.165838,0.192926,2.539957
min,0.612885,12.000000,1.000000,17.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.257702,50.000000,3.000000,47.000000


In [12]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [13]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [14]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
39,1,32,1,41.0
6132,0,28,1,30.0
5986,0,44,1,38.0
7682,0,34,1,38.0
4910,1,31,1,40.0


In [15]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [16]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

2021-10-11 15:35:49.986729: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [17]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 2,433
Trainable params: 2,433
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

2021-10-11 15:36:46.264650: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
209/209 [==============================] - 2s 5ms/step - loss: 5.7630 - mae: 1.3446 - mse: 5.7630 - val_loss: 1.7706 - val_mae: 1.0670 - val_mse: 1.7706
Epoch 2/10
209/209 [==============================] - 1s 3ms/step - loss: 1.3427 - mae: 0.9071 - mse: 1.3427 - val_loss: 1.1985 - val_mae: 0.8484 - val_mse: 1.1985
Epoch 3/10
209/209 [==============================] - 1s 3ms/step - loss: 1.3032 - mae: 0.8921 - mse: 1.3032 - val_loss: 1.7086 - val_mae: 1.0302 - val_mse: 1.7086
Epoch 4/10
209/209 [==============================] - 1s 3ms/step - loss: 1.2778 - mae: 0.8856 - mse: 1.2778 - val_loss: 1.1593 - val_mae: 0.8355 - val_mse: 1.1593
Epoch 5/10
209/209 [==============================] - 1s 3ms/step - loss: 1.2724 - mae: 0.8816 - mse: 1.2724 - val_loss: 1.3526 - val_mae: 0.9082 - val_mse: 1.3526
Epoch 6/10
209/209 [==============================] - ETA: 0s - loss: 1.2640 - mae: 0.8754 - mse: 1.264 - 1s 4ms/step - loss: 1.2561 - mae: 0.8740 - mse: 1.2561 - val_loss: 1.1529 

In [20]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [21]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.892152
Actual val:  7.1870697412

Predicted val:  7.7843285
Actual val:  9.87450471498

Predicted val:  7.690989
Actual val:  7.3744626639

Predicted val:  6.7109127
Actual val:  5.8135898489399995

Predicted val:  7.731707
Actual val:  7.87491199864

Predicted val:  7.3143396
Actual val:  6.5256829552

Predicted val:  7.008233
Actual val:  7.45823832346

Predicted val:  8.20076
Actual val:  7.43619209726

Predicted val:  7.463129
Actual val:  7.90577671532

Predicted val:  6.763049
Actual val:  6.9996768185



In [22]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [23]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [27]:
try:
  import google.colab
  !pip install --upgrade witwidget
except:
  pass

In [28]:
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [ ]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…